In [1]:
#import the dataset and show the first 5 lines of CRSP 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Handle compustat data
compu = pd.read_csv("/content/drive/My Drive/FINA3327_Final/compu_uodated.csv")
compu = compu.rename(columns={'LPERMNO':'PERMNO','datadate':'date'},inplace = False)
compu = compu[['PERMNO','date','fyear','at','cogs','revt','seq']]
compu['date'] = compu['date']//100
compu.head()

,PERMNO,date,fyear,at,cogs,revt,seq
0,25881,197012,1970.0,33.450,30.529,45.335,10.544
1,25881,197112,1971.0,29.330,33.973,47.033,8.382
2,25881,197212,1972.0,19.907,22.702,34.362,7.021
3,25881,197312,1973.0,21.771,24.704,37.750,8.567
4,25881,197412,1974.0,25.638,36.646,50.325,10.257


In [4]:
#Handle CRSP data
crsp = pd.read_csv("/content/drive/My Drive/FINA3327_Final/crsp.csv")
crsp['fyear'] = crsp['date']//10000.0
crsp['date'] = crsp['date']//100
crsp['RET'] = crsp['RET'].replace('C',np.nan)
crsp['RET'] = crsp['RET'].replace('B',np.nan)
crsp['mkt_cap'] = crsp['PRC']*crsp['SHROUT']
crsp.head()

,PERMNO,date,PERMCO,PRC,RET,SHROUT,fyear,mkt_cap
0,10000,198512,7952,NaN,NaN,NaN,1985,NaN
1,10000,198601,7952,-4.3750,NaN,3680.0,1986,-16100.0
2,10000,198602,7952,-3.2500,-0.257143,3680.0,1986,-11960.0
3,10000,198603,7952,-4.4375,0.365385,3680.0,1986,-16330.0
4,10000,198604,7952,-4.0000,-0.098592,3793.0,1986,-15172.0


In [5]:
#Merge compustat and crsp
whole_data = pd.merge(crsp, compu, 'left')
whole_data['LPERMNO'] = whole_data['PERMNO'] 
whole_data = whole_data.groupby(whole_data['PERMNO']).ffill()
whole_data = whole_data.rename({'LPERMNO':'PERMNO'})
whole_data.head()

,date,PERMCO,PRC,RET,SHROUT,fyear,mkt_cap,at,cogs,revt,seq,LPERMNO
0,198512,7952,NaN,NaN,NaN,1985,NaN,NaN,NaN,NaN,NaN,10000
1,198601,7952,-4.3750,NaN,3680.0,1986,-16100.0,NaN,NaN,NaN,NaN,10000
2,198602,7952,-3.2500,-0.257143,3680.0,1986,-11960.0,NaN,NaN,NaN,NaN,10000
3,198603,7952,-4.4375,0.365385,3680.0,1986,-16330.0,NaN,NaN,NaN,NaN,10000
4,198604,7952,-4.0000,-0.098592,3793.0,1986,-15172.0,NaN,NaN,NaN,NaN,10000


In [7]:
#Drop rows where mkt_cap(ie. PRC) is negative because there is no trade that month
indexNames = whole_data[ whole_data['mkt_cap'] < 0 ].index
whole_data = whole_data.drop(indexNames, inplace=False)
whole_data

,date,PERMCO,PRC,RET,SHROUT,fyear,mkt_cap,at,cogs,revt,seq,LPERMNO
28,198609,7953,6.37500,-0.003077,991.0,1986,6.317625e+03,12.242,19.565,21.46,5.432,10001
29,198610,7953,6.62500,0.039216,991.0,1986,6.565375e+03,12.242,19.565,21.46,5.432,10001
30,198611,7953,7.00000,0.056604,991.0,1986,6.937000e+03,12.242,19.565,21.46,5.432,10001
31,198612,7953,7.00000,0.015000,991.0,1986,6.937000e+03,12.242,19.565,21.46,5.432,10001
32,198701,7953,6.75000,-0.035714,991.0,1987,6.689250e+03,12.242,19.565,21.46,5.432,10001
...,...,...,...,...,...,...,...,...,...,...,...,...
4477710,202008,53453,498.32001,0.741452,931809.0,2020,4.643391e+08,34309.000,18402.000,24578.00,6618.000,93436
4477711,202009,53453,429.01001,-0.139087,948000.0,2020,4.067015e+08,34309.000,18402.000,24578.00,6618.000,93436
4477712,202010,53453,388.04001,-0.095499,947901.0,2020,3.678235e+08,34309.000,18402.000,24578.00,6618.000,93436
4477713,202011,53453,567.59998,0.462736,947901.0,2020,5.380286e+08,34309.000,18402.000,24578.00,6618.000,93436


In [8]:
#Drop data which has NAN in any coloums
whole_data = whole_data.dropna(axis=0,how='any')
whole_data

,date,PERMCO,PRC,RET,SHROUT,fyear,mkt_cap,at,cogs,revt,seq,LPERMNO
28,198609,7953,6.37500,-0.003077,991.0,1986,6.317625e+03,12.242,19.565,21.46,5.432,10001
29,198610,7953,6.62500,0.039216,991.0,1986,6.565375e+03,12.242,19.565,21.46,5.432,10001
30,198611,7953,7.00000,0.056604,991.0,1986,6.937000e+03,12.242,19.565,21.46,5.432,10001
31,198612,7953,7.00000,0.015000,991.0,1986,6.937000e+03,12.242,19.565,21.46,5.432,10001
32,198701,7953,6.75000,-0.035714,991.0,1987,6.689250e+03,12.242,19.565,21.46,5.432,10001
...,...,...,...,...,...,...,...,...,...,...,...,...
4477710,202008,53453,498.32001,0.741452,931809.0,2020,4.643391e+08,34309.000,18402.000,24578.00,6618.000,93436
4477711,202009,53453,429.01001,-0.139087,948000.0,2020,4.067015e+08,34309.000,18402.000,24578.00,6618.000,93436
4477712,202010,53453,388.04001,-0.095499,947901.0,2020,3.678235e+08,34309.000,18402.000,24578.00,6618.000,93436
4477713,202011,53453,567.59998,0.462736,947901.0,2020,5.380286e+08,34309.000,18402.000,24578.00,6618.000,93436


In [9]:
#Calculate profitability and BM_ratio
whole_data['Profitability'] = (whole_data['revt']-whole_data['cogs'])/whole_data['at']
whole_data['BM_ratio'] = (whole_data['seq']/whole_data['mkt_cap'])
whole_data['rankPro'] = whole_data['Profitability'].groupby(whole_data['date']).rank(ascending=False,method='dense')
whole_data['rankBM'] = whole_data['BM_ratio'].groupby(whole_data['date']).rank(ascending=False,method='dense')
whole_data

,date,PERMCO,PRC,RET,SHROUT,fyear,mkt_cap,at,cogs,revt,seq,LPERMNO,Profitability,BM_ratio,rankPro,rankBM
28,198609,7953,6.37500,-0.003077,991.0,1986,6.317625e+03,12.242,19.565,21.46,5.432,10001,0.154795,0.000860,2151.0,913.0
29,198610,7953,6.62500,0.039216,991.0,1986,6.565375e+03,12.242,19.565,21.46,5.432,10001,0.154795,0.000827,2181.0,948.0
30,198611,7953,7.00000,0.056604,991.0,1986,6.937000e+03,12.242,19.565,21.46,5.432,10001,0.154795,0.000783,2114.0,1013.0
31,198612,7953,7.00000,0.015000,991.0,1986,6.937000e+03,12.242,19.565,21.46,5.432,10001,0.154795,0.000783,2322.0,1310.0
32,198701,7953,6.75000,-0.035714,991.0,1987,6.689250e+03,12.242,19.565,21.46,5.432,10001,0.154795,0.000812,2216.0,944.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4477710,202008,53453,498.32001,0.741452,931809.0,2020,4.643391e+08,34309.000,18402.000,24578.00,6618.000,93436,0.180011,0.000014,1889.0,4106.0
4477711,202009,53453,429.01001,-0.139087,948000.0,2020,4.067015e+08,34309.000,18402.000,24578.00,6618.000,93436,0.180011,0.000016,1880.0,4093.0
4477712,202010,53453,388.04001,-0.095499,947901.0,2020,3.678235e+08,34309.000,18402.000,24578.00,6618.000,93436,0.180011,0.000018,1867.0,4069.0
4477713,202011,53453,567.59998,0.462736,947901.0,2020,5.380286e+08,34309.000,18402.000,24578.00,6618.000,93436,0.180011,0.000012,1866.0,4076.0


In [10]:
#Define strategy_rule
whole_data['strategy_rule'] = (whole_data['rankPro']<=50) | (whole_data['rankBM']<=50)
whole_data['strategy_rule'] = whole_data['strategy_rule'].astype(int) # convert T/F to 1/0
whole_data

,date,PERMCO,PRC,RET,SHROUT,fyear,mkt_cap,at,cogs,revt,seq,LPERMNO,Profitability,BM_ratio,rankPro,rankBM,strategy_rule
28,198609,7953,6.37500,-0.003077,991.0,1986,6.317625e+03,12.242,19.565,21.46,5.432,10001,0.154795,0.000860,2151.0,913.0,0
29,198610,7953,6.62500,0.039216,991.0,1986,6.565375e+03,12.242,19.565,21.46,5.432,10001,0.154795,0.000827,2181.0,948.0,0
30,198611,7953,7.00000,0.056604,991.0,1986,6.937000e+03,12.242,19.565,21.46,5.432,10001,0.154795,0.000783,2114.0,1013.0,0
31,198612,7953,7.00000,0.015000,991.0,1986,6.937000e+03,12.242,19.565,21.46,5.432,10001,0.154795,0.000783,2322.0,1310.0,0
32,198701,7953,6.75000,-0.035714,991.0,1987,6.689250e+03,12.242,19.565,21.46,5.432,10001,0.154795,0.000812,2216.0,944.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4477710,202008,53453,498.32001,0.741452,931809.0,2020,4.643391e+08,34309.000,18402.000,24578.00,6618.000,93436,0.180011,0.000014,1889.0,4106.0,0
4477711,202009,53453,429.01001,-0.139087,948000.0,2020,4.067015e+08,34309.000,18402.000,24578.00,6618.000,93436,0.180011,0.000016,1880.0,4093.0,0
4477712,202010,53453,388.04001,-0.095499,947901.0,2020,3.678235e+08,34309.000,18402.000,24578.00,6618.000,93436,0.180011,0.000018,1867.0,4069.0,0
4477713,202011,53453,567.59998,0.462736,947901.0,2020,5.380286e+08,34309.000,18402.000,24578.00,6618.000,93436,0.180011,0.000012,1866.0,4076.0,0


In [11]:
#Portfolio construction
def eq_weight(df):
    stock_number = df['strategy_rule'].sum()
    # our portfolio is re-constructed every month
    try:
        df['eq_weight'] = df['strategy_rule']/stock_number
    except:
        weights = np.nan
    return df
def val_weight(df):
    total_mv = df.loc[df['strategy_rule']==1,'mkt_cap'].sum()
    try:
        df['val_weight']= df.loc[df['strategy_rule']==1,'mkt_cap']/total_mv
    except:
        weights = np.nan
    return df

In [12]:
def sort_rank_Pro(df):
    sorted_list_Pro = df.sort_values(by=['rankPro','date'],ascending=[True,True])
    return sorted_list_Pro
def sort_rank_BM(df):
    sorted_list_BM = df.sort_values(by=['rankBM','date'],ascending=[True,True])
    return sorted_list_BM

In [14]:
whole_data_w = whole_data.groupby('date').apply(eq_weight)
whole_data_w = whole_data_w.groupby('date').apply(val_weight) 
whole_data_w.groupby('date').apply(sort_rank_Pro)

# we define a new dataframe "show" here to present the result of weighting
show = whole_data_w[(whole_data_w['strategy_rule']==1.0) 
          & (whole_data_w['date'] == 200703) 
          & (whole_data_w['fyear'] == 2007)]
show.sort_values(by=['rankPro'],ascending=[True]).head(10)

,date,PERMCO,PRC,RET,SHROUT,fyear,mkt_cap,at,cogs,revt,seq,LPERMNO,Profitability,BM_ratio,rankPro,rankBM,strategy_rule,eq_weight,val_weight
2694486,200703,22037,66.48,0.076599,21400.0,2007,1422672.00,9.044,0.000,184.864,8.853,75622,20.440513,6.222798e-06,1.0,4964.0,1,0.009901,0.025535
2376806,200703,21550,41.23,0.035138,14579.0,2007,601092.17,5.370,0.000,61.013,4.998,64805,11.361825,8.314865e-06,2.0,4957.0,1,0.009901,0.010789
2299664,200703,21389,14.76,0.006098,46609.0,2007,687948.84,6.574,0.000,66.407,1.439,62359,10.101460,2.091725e-06,3.0,4975.0,1,0.009901,0.012348
2302782,200703,21560,31.24,-0.003936,46609.0,2007,1456065.16,26.481,0.000,135.318,21.823,62391,5.110003,1.498765e-05,4.0,4946.0,1,0.009901,0.026134
2340067,200703,21298,37.25,-0.008253,9191.0,2007,342364.75,7.204,0.000,31.079,0.036,63706,4.314131,1.051510e-07,5.0,4981.0,1,0.009901,0.006145
3564392,200703,14845,46.87,-0.192871,17875.0,2007,837801.25,100.002,89.545,374.190,60.197,84010,2.846393,7.185117e-05,6.0,4812.0,1,0.009901,0.015037
3100297,200703,29794,10.94,0.037951,16274.0,2007,178037.56,37.282,18.277,117.467,27.734,79065,2.660533,1.557761e-04,7.0,4469.0,1,0.009901,0.003196
3866397,200703,17399,19.46,0.112628,12432.0,2007,241926.72,66.012,30.659,185.089,29.979,87404,2.339423,1.239177e-04,8.0,4607.0,1,0.009901,0.004342
3787299,200703,16332,17.80,-0.039396,49551.0,2007,882007.80,228.961,295.846,806.038,115.694,86489,2.228292,1.311712e-04,9.0,4580.0,1,0.009901,0.015831
3391170,200703,13796,1.85,-0.075000,8200.0,2007,15170.00,48.585,31.756,133.428,26.975,81728,2.092662,1.778181e-03,10.0,327.0,1,0.009901,0.000272


In [15]:
#Calculate return
def calculate_return(df):
    df['RET'] = df['RET'].astype(float)
    df['eq_weighted_r'] = df['eq_weight'] * df['RET']
    df['val_weighted_r'] = df['val_weight'] * df['RET']
    eq_return = pd.pivot_table(df,index='date',values='eq_weighted_r',aggfunc=np.sum)
    val_return = pd.pivot_table(df,index='date',values='val_weighted_r',aggfunc=np.sum)
    return_dataset = pd.concat([eq_return,val_return],axis=1)
    return return_dataset

In [16]:
def shift_return(df):
    df['val_weight'] = df['val_weight'].shift(1)
    return df

In [17]:
# shift return
portfolio = whole_data_w[whole_data_w.columns].groupby('LPERMNO').apply(shift_return)
portfolio = portfolio.dropna(axis=0,how='any')
portfolio.head()

,date,PERMCO,PRC,RET,SHROUT,fyear,mkt_cap,at,cogs,revt,seq,LPERMNO,Profitability,BM_ratio,rankPro,rankBM,strategy_rule,eq_weight,val_weight
2442,198502,6398,6.000,0.200000,3570.0,1985,21420.000,16.267,8.171,32.007,8.962,10015,1.465298,0.000418,11.0,2062.0,1,0.010000,0.000692
2443,198503,6398,5.500,-0.083333,3563.0,1985,19596.500,16.267,8.171,32.007,8.962,10015,1.465298,0.000457,12.0,2039.0,1,0.010000,0.000810
2444,198504,6398,6.125,0.113636,3563.0,1985,21823.375,16.267,8.171,32.007,8.962,10015,1.465298,0.000411,12.0,2220.0,1,0.009901,0.000783
2445,198505,6398,5.875,-0.040816,3563.0,1985,20932.625,16.267,8.171,32.007,8.962,10015,1.465298,0.000428,12.0,2181.0,1,0.010000,0.000904
2446,198506,6398,6.500,0.106383,3985.0,1985,25902.500,16.267,8.171,32.007,8.962,10015,1.465298,0.000346,11.0,2421.0,1,0.010000,0.000910


In [18]:
#Calculate return
return_dataset = calculate_return(portfolio)
return_dataset.head(5)

,eq_weighted_r,val_weighted_r
date,,
196001,0.025000,0.025000
196002,-0.018018,-0.018018
196003,-0.050459,-0.050459
196004,0.009756,0.009756
196005,-0.057971,-0.057971


In [19]:
#Cumulative return
final = return_dataset.copy()
final['eq_weighted_R'] = return_dataset['eq_weighted_r']+1
final['val_weighted_R'] = return_dataset['val_weighted_r']+1
final['eq_cum_R'] = final['eq_weighted_R'].cumprod()
final['val_cum_R'] = final['val_weighted_R'].cumprod()
final.head()

,eq_weighted_r,val_weighted_r,eq_weighted_R,val_weighted_R,eq_cum_R,val_cum_R
date,,,,,,
196001,0.025000,0.025000,1.025000,1.025000,1.025000,1.025000
196002,-0.018018,-0.018018,0.981982,0.981982,1.006532,1.006532
196003,-0.050459,-0.050459,0.949541,0.949541,0.955743,0.955743
196004,0.009756,0.009756,1.009756,1.009756,0.965067,0.965067
196005,-0.057971,-0.057971,0.942029,0.942029,0.909121,0.909121


In [20]:
#Export return table
final.to_excel('3327 final Profit Combine.xlsx', sheet_name = 'return')

In [21]:
#Data analysis
reg_raw = pd.read_excel("/content/drive/My Drive/FINA3327_Final/stat_data_combine.xlsx")
reg_raw.head()

,Date,eq_weighted_r,val_weighted_r,eq_cum_R,val_cum_R,market_mon_r,market_mon_R,mkt_cum-r,eq_cum_r*,val_cum_r*,Mkt-rf,SMB,HML,Momentum,rf,eq_r-rf,val_r-rf
0,196001,0.025000,0.025000,1.025000,1.025000,-0.0665,0.9335,0.933500,1.025000,1.025000,-0.0698,0.0205,0.0269,-0.0349,0.0033,0.021700,0.021700
1,196002,-0.018018,-0.018018,1.006532,1.006532,0.0146,1.0146,0.947129,1.006532,1.006532,0.0117,0.0056,-0.0203,0.0386,0.0029,-0.020918,-0.020918
2,196003,-0.050459,-0.050459,0.955743,0.955743,-0.0128,0.9872,0.935006,0.955743,0.955743,-0.0163,-0.0047,-0.0284,0.0143,0.0035,-0.053959,-0.053959
3,196004,0.009756,0.009756,0.965067,0.965067,-0.0152,0.9848,0.920794,0.965067,0.965067,-0.0171,0.0039,-0.0237,0.0281,0.0019,0.007856,0.007856
4,196005,-0.057971,-0.057971,0.909121,0.909121,0.0339,1.0339,0.952009,0.909121,0.909121,0.0312,0.0127,-0.0372,0.0481,0.0027,-0.060671,-0.060671


In [28]:
#Equal_weighted Carhart
y_e_Ca = reg_raw[['eq_r-rf']]
x_Ca = reg_raw[['HML','SMB','Mkt-rf','Momentum']]
x_Ca.head()

,HML,SMB,Mkt-rf,Momentum
0,0.0269,0.0205,-0.0698,-0.0349
1,-0.0203,0.0056,0.0117,0.0386
2,-0.0284,-0.0047,-0.0163,0.0143
3,-0.0237,0.0039,-0.0171,0.0281
4,-0.0372,0.0127,0.0312,0.0481


In [29]:
y_e_Ca.head()

,eq_r-rf
0,0.021700
1,-0.020918
2,-0.053959
3,0.007856
4,-0.060671


In [30]:
import statsmodels.api as sm
x1 = sm.add_constant(x_Ca)
lm = sm.OLS(y_e_Ca.astype(float),x1.astype(float)).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                eq_r-rf   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.772
Method:                 Least Squares   F-statistic:                     578.6
Date:                Tue, 20 Apr 2021   Prob (F-statistic):          3.38e-217
Time:                        07:13:23   Log-Likelihood:                 1489.8
No. Observations:                 685   AIC:                            -2970.
Df Residuals:                     680   BIC:                            -2947.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0030      0.001     -2.740      0.0

In [31]:
#Value weighted carhart
y_v_Ca = reg_raw['val_r-rf']

In [32]:
lm = sm.OLS(y_v_Ca.astype(float),x1.astype(float)).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:               val_r-rf   R-squared:                       0.692
Model:                            OLS   Adj. R-squared:                  0.690
Method:                 Least Squares   F-statistic:                     381.4
Date:                Tue, 20 Apr 2021   Prob (F-statistic):          4.20e-172
Time:                        07:14:21   Log-Likelihood:                 1445.0
No. Observations:                 685   AIC:                            -2880.
Df Residuals:                     680   BIC:                            -2857.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0024      0.001      2.071      0.0

In [ ]:
#equal weighted CAPM

In [33]:
y_e_CA = reg_raw[['eq_r-rf']]
x_CA = reg_raw[['Mkt-rf']]

In [34]:
x2 = sm.add_constant(x_CA)
lm = sm.OLS(y_e_CA.astype(float),x2.astype(float)).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                eq_r-rf   R-squared:                       0.605
Model:                            OLS   Adj. R-squared:                  0.604
Method:                 Least Squares   F-statistic:                     1046.
Date:                Tue, 20 Apr 2021   Prob (F-statistic):          7.10e-140
Time:                        07:16:25   Log-Likelihood:                 1300.1
No. Observations:                 685   AIC:                            -2596.
Df Residuals:                     683   BIC:                            -2587.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0034      0.001     -2.443      0.0

In [ ]:
#value weighted CAPM

In [36]:
y_v_CA = reg_raw[['val_r-rf']]

In [37]:
lm = sm.OLS(y_v_CA.astype(float),x2.astype(float)).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:               val_r-rf   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.676
Method:                 Least Squares   F-statistic:                     1425.
Date:                Tue, 20 Apr 2021   Prob (F-statistic):          2.55e-169
Time:                        07:16:59   Log-Likelihood:                 1428.0
No. Observations:                 685   AIC:                            -2852.
Df Residuals:                     683   BIC:                            -2843.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0021      0.001      1.811      0.0